In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #130
!cp -r '/content/gdrive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)' /content

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/taesungp/contrastive-unpaired-translation CUT
%cd CUT

Cloning into 'CUT'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 256 (delta 9), reused 13 (delta 5), pack-reused 238
Receiving objects: 100% (256/256), 17.90 MiB | 26.23 MiB/s, done.
Resolving deltas: 100% (125/125), done.
/content/CUT


In [1]:
%cd CUT

/content/CUT


In [ ]:
import glob
uncover_img_dirs = sorted(glob.glob('/content/In-Bed-Human-Pose-Estimation(VIP-CUP)/train/*/IR/uncover/*'))
cover1_img_dirs = sorted(glob.glob('/content/In-Bed-Human-Pose-Estimation(VIP-CUP)/train/*/IR/cover1/*'))
cover2_img_dirs = sorted(glob.glob('/content/In-Bed-Human-Pose-Estimation(VIP-CUP)/train/*/IR/cover2/*'))

import shutil, os
!rm -rf 'datasets/inbed/trainA'
!rm -rf 'datasets/inbed/trainB'
!rm -rf 'datasets/inbed/testA'
!rm -rf 'datasets/inbed/testB'

!mkdir -p 'datasets/inbed/trainA'
!mkdir -p 'datasets/inbed/trainB'
!mkdir -p 'datasets/inbed/testA'
!mkdir -p 'datasets/inbed/testB'

def copy_imgs(img_dirs, target_dir, A_or_B='A', train_test_ratio= 0.8):
  n_images= len(img_dirs)
  n_train= int(n_images*0.8)
  n_test= n_images- n_train

  for i in range(n_images):
    img_dir= img_dirs[i]
    img_name= (6-len(str(i+1)))*'0'+str(i+1)+'.jpg'
    if i<n_train:target_img_dir= f'{target_dir}/train{A_or_B}/{img_name}'
    else:target_img_dir= f'{target_dir}/test{A_or_B}/{img_name}'
    
    print(target_img_dir)
    shutil.copyfile(img_dir, target_img_dir)

copy_imgs(cover1_img_dirs, 'datasets/inbed', 'A', 0.8)
copy_imgs(uncover_img_dirs, 'datasets/inbed', 'B', 0.2)

datasets/inbed/trainA/000263.jpg
datasets/inbed/trainA/000264.jpg
datasets/inbed/trainA/000265.jpg
datasets/inbed/trainA/000266.jpg
datasets/inbed/trainA/000267.jpg
datasets/inbed/trainA/000268.jpg
datasets/inbed/trainA/000269.jpg
datasets/inbed/trainA/000270.jpg
datasets/inbed/trainA/000271.jpg
datasets/inbed/trainA/000272.jpg
datasets/inbed/trainA/000273.jpg
datasets/inbed/trainA/000274.jpg
datasets/inbed/trainA/000275.jpg
datasets/inbed/trainA/000276.jpg
datasets/inbed/trainA/000277.jpg
datasets/inbed/trainA/000278.jpg
datasets/inbed/trainA/000279.jpg
datasets/inbed/trainA/000280.jpg
datasets/inbed/trainA/000281.jpg
datasets/inbed/trainA/000282.jpg
datasets/inbed/trainA/000283.jpg
datasets/inbed/trainA/000284.jpg
datasets/inbed/trainA/000285.jpg
datasets/inbed/trainA/000286.jpg
datasets/inbed/trainA/000287.jpg
datasets/inbed/trainA/000288.jpg
datasets/inbed/trainA/000289.jpg
datasets/inbed/trainA/000290.jpg
datasets/inbed/trainA/000291.jpg
datasets/inbed/trainA/000292.jpg
datasets/i

In [ ]:
%%capture
!pip install -r requirements.txt
!pip install torch==1.2.0 torchvision==0.4.0

In [2]:
%%writefile train.py

import time
import torch
from options.train_options import TrainOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer
import shutil
import os


if __name__ == '__main__':
    opt = TrainOptions().parse()   # get training options
    dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
    dataset_size = len(dataset)    # get the number of images in the dataset.

    model = create_model(opt)      # create a model given opt.model and other options
    print('The number of training images = %d' % dataset_size)

    visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
    opt.visualizer = visualizer
    total_iters = 0                # the total number of training iterations

    optimize_time = 0.1

    times = []
    for epoch in range(opt.epoch_count, opt.n_epochs + opt.n_epochs_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
        epoch_start_time = time.time()  # timer for entire epoch
        iter_data_time = time.time()    # timer for data loading per iteration
        epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch
        visualizer.reset()              # reset the visualizer: make sure it saves the results to HTML at least once every epoch

        dataset.set_epoch(epoch)
        for i, data in enumerate(dataset):  # inner loop within one epoch
            iter_start_time = time.time()  # timer for computation per iteration
            if total_iters % opt.print_freq == 0:
                t_data = iter_start_time - iter_data_time

            batch_size = data["A"].size(0)
            total_iters += batch_size
            epoch_iter += batch_size
            if len(opt.gpu_ids) > 0:
                torch.cuda.synchronize()
            optimize_start_time = time.time()
            if epoch == opt.epoch_count and i == 0:
                model.data_dependent_initialize(data)
                model.setup(opt)               # regular setup: load and print networks; create schedulers
                model.parallelize()
            model.set_input(data)  # unpack data from dataset and apply preprocessing
            model.optimize_parameters()   # calculate loss functions, get gradients, update network weights
            if len(opt.gpu_ids) > 0:
                torch.cuda.synchronize()
            optimize_time = (time.time() - optimize_start_time) / batch_size * 0.005 + 0.995 * optimize_time

            if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
                save_result = total_iters % opt.update_html_freq == 0
                model.compute_visuals()
                visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

            if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
                losses = model.get_current_losses()
                visualizer.print_current_losses(epoch, epoch_iter, losses, optimize_time, t_data)
                if opt.display_id is None or opt.display_id > 0:
                    visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

            if total_iters % opt.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
                print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
                print(opt.name)  # it's useful to occasionally show the experiment name on console
                save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
                model.save_networks(save_suffix)

            iter_data_time = time.time()

        if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
            print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
            model.save_networks('latest')
            model.save_networks(epoch)

            drive_path= f'/content/gdrive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/CUT_unpaired/checkpoints/{opt.name}'
            if os.path.isdir(drive_path):shutil.rmtree(drive_path)
            shutil.copytree(f'checkpoints/{opt.name}', drive_path)
            print(f'saving to drive (checkpoints) : {drive_path}')


        drive_path= f'/content/gdrive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/CUT_unpaired/checkpoints/{opt.name}'
        if not os.path.isdir(drive_path):os.mkdir(drive_path)
        img_saved_path = f'{drive_path}/web'
        try:shutil.rmtree(img_saved_path)
        except:pass
        shutil.copytree(f'checkpoints/{opt.name}/web', img_saved_path)
        print(f'saving to drive (only images) : {img_saved_path}')

        print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time))
        model.update_learning_rate()                     # update learning rates at the end of every epoch.

Overwriting train.py


In [3]:
!cp -r '/content/gdrive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/CUT_unpaired/checkpoints' .

In [4]:
!ls checkpoints

InbedPose_CUT  InbedPose_FastCUT


In [5]:
#!rm -rf '/content/gdrive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/CUT_unpaired/'
#!mkdir -p '/content/gdrive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/CUT_unpaired/checkpoints'

In [ ]:
!python train.py --dataroot ./datasets/inbed --name InbedPose_CUT --CUT_mode CUT --continue_train

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: ./datasets/inbed              	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: None                          
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost     

In [ ]:
#!python train.py --dataroot ./datasets/inbed --name InbedPose_FastCUT --CUT_mode FastCUT